In [1]:
import torch
#force to use CPU
# torch.cuda.is_available = lambda : False
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import pathlib
import time
import os

temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

import utils
display = utils.notebook_init()

import urllib.request
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
import cv2

import torchvision.transforms as transforms

import torch.nn.functional as F
import imutils as im
from utils.dataloaders import LoadImages
from utils.general import check_img_size
from utils.augmentations import classify_transforms

car_model = torch.hub.load('C:\Project\yolov5', 'custom', path='C:\Project\yolov5\my_weight\car_model_600.pt', source='local')

license_model = torch.hub.load('C:\Project\yolov5', 'custom', path='C:\Project\yolov5\my_weight\license_object_detection_500_real.pt', source='local')

model = torch.hub.load('C:\Project\yolov5', 'custom', path='C:\Project\yolov5\my_weight\letter_object_nano_all_real_final.pt', source='local')

classification_model = torch.hub.load('C:\Project\yolov5', 'custom', path='C:\Project\yolov5\my_weight\letter_class_real_100epoch_big.pt', source='local')

alphabet_dict = {
    "0E01": "ก",
    "0E02": "ข",
    "0E04": "ค",
    "0E06": "ฆ",
    "0E07": "ง",
    "0E08": "จ",
    "0E09": "ฉ",
    "0E0A": "ช",
    "0E0C": "ฌ",
    "0E0D": "ญ",
    "0E0E": "ฎ",
    "0E10": "ฐ",
    "0E12": "ฒ",
    "0E13": "ณ",
    "0E14": "ด",
    "0E15": "ต",
    "0E16": "ถ",
    "0E17": "ท",
    "0E18": "ธ",
    "0E19": "น",
    "0E1A": "บ",
    "0E1B": "ป",
    "0E1C": "ผ",
    "0E1E": "พ",
    "0E1F": "ฟ",
    "0E20": "ภ",
    "0E21": "ม",
    "0E22": "ย",
    "0E23": "ร",
    "0E25": "ล",
    "0E27": "ว",
    "0E28": "ศ",
    "0E29": "ษ",
    "0E2A": "ส",
    "0E2B": "ห",
    "0E2C": "ฬ",
    "0E2D": "อ",
    "0E2E": "ฮ",
    "n0": "0",
    "n1":"1",
    "n2":"2",
    "n3":"3",
    "n4":"4",
    "n5":"5",
    "n6":"6",
    "n7":"7",
    "n8":"8",
    "n9":"9"
}
number_list = ["n1","n2","n3","n4","n5","n6","n7","n8","n9","n0"]

YOLOv5  2022-12-7 Python-3.9.12 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)


Setup complete  (16 CPUs, 15.8 GB RAM, 390.2/465.1 GB disk)


YOLOv5  2022-12-7 Python-3.9.12 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 
YOLOv5  2022-12-7 Python-3.9.12 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 
YOLOv5  2022-12-7 Python-3.9.12 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 1761871 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 
YOLOv5  2022-12-7 Python-3.9.12 torch-1.12.1+cu116 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)

Fusing layers... 
Model summary: 117 layers, 4228176 parameters, 0 gradients, 10.4 GFLOPs
WARNING  YOLOv5 ClassificationModel is not yet AutoShape compatible. You must pass torch tensors in BCHW to this model, i.e. shape(1,3,224,224).


In [3]:
import numpy as np
import cv2
from mss import mss
from PIL import Image, ImageFont, ImageDraw
import io

sct = mss()
global_count=0

while 1:
    w, h = 1920, 1080
    sw, sh = 0,0
    monitor = {'top': sh, 'left': sw, 'width': w-sw, 'height': h-sh}
    sct_img = sct.grab(monitor)
    img = Image.frombytes('RGB', sct_img.size, sct_img.rgb)
    cv2.namedWindow("test", cv2.WINDOW_NORMAL)
    # Resize this window
    cv2.resizeWindow("test", int((w-sw)*0.6), int((h-sh)*0.6))
    cv2.imshow('test', cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR))

    results = car_model(img)
    dfResults = results.pandas().xyxy[0]
    #check if there is at least one car detected
    if len(dfResults) > 0:
      image = img
      image_arr = np.array(image)
      #choose car start at 0
      letter_index=0
      x1=int(results.xyxy[0][letter_index][0])
      x2=int(results.xyxy[0][letter_index][2])
      y1=int(results.xyxy[0][letter_index][1])
      y2=int(results.xyxy[0][letter_index][3])
      image_arr2 = image_arr[y1:y2, x1:x2]
      save_img_car = Image.fromarray(image_arr2)
      save_img_car = save_img_car.convert('RGB')
    #   save_img_car.save('./my_output/'+str(global_count)+'_crop_car.jpg')
    #   global_count+=1

      results = license_model(image_arr2)
      dfResults = results.pandas().xyxy[0]
      #check if there is at least one license plate detected
      if len(dfResults) > 0:
          #gray scale
          image = Image.fromarray(image_arr2)
          image = image.convert('L')
          image_arr = np.array(image)
          #choose license plate start at 0
          letter_index=0
          #check if detected plate width is > height
          while len(results.xyxy[0]) > letter_index:
            x1=int(results.xyxy[0][letter_index][0])
            x2=int(results.xyxy[0][letter_index][2])
            y1=int(results.xyxy[0][letter_index][1])
            y2=int(results.xyxy[0][letter_index][3])
            if (x2-x1) > (y2-y1):
              image_arr2 = image_arr[y1:y2, x1:x2]
              break
            letter_index+=1

          def image_resize(image, width = None, height = None, inter = cv2.INTER_CUBIC):
              dim = None
              (h, w) = image.shape[:2]
              if width is None and height is None:
                  return image
              if width is None:
                  r = height / float(h)
                  dim = (int(w * r), height)
              else:
                  r = width / float(w)
                  dim = (width, int(h * r))
              resized = cv2.resize(image, dim, interpolation = inter)
              return resized
          save_img = image_resize(image_arr2, height = 100)
          save_img = cv2.fastNlMeansDenoising(save_img, None, 20, 7, 21)
          kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
          save_img = cv2.filter2D(save_img, -1, kernel)
          cv2.imwrite('./crop_img2.jpg',save_img)

          #letter_province
          results = model(save_img)
          dfResults = results.pandas().xyxy[0]

          count_letter=len(results.xyxy[0])
          if(torch.cuda.is_available()):
              xmin_sort=torch.tensor([]).to('cuda')
          else:
              xmin_sort=torch.tensor([]).to('cpu')
          for i in range(count_letter):
              #convert from dim0 to dim1
              if(torch.cuda.is_available()):
                  xmin=torch.tensor([results.xyxy[0][i][0]]).to('cuda')
              else:
                  xmin=torch.tensor([results.xyxy[0][i][0]]).to('cpu')
              xmin_sort=torch.cat((xmin_sort,xmin),0)
          xmin_sorted, _ = torch.sort(xmin_sort)


          #Letter_object detection extract

          if len(results.pandas().xyxy[0]) > 0:
              image_arr = save_img
              #image_arr[y1:y2, x1:x2]
              license_string_list=[]
              string_count=0
              letter_index=0
              while letter_index<count_letter:
                my_index=0
                while my_index < count_letter:
                  if xmin_sorted[letter_index] == results.xyxy[0][my_index][0]:
                    if results.xyxy[0][my_index][5] < 1:
                      x1=int(results.xyxy[0][my_index][0])
                      x2=int(results.xyxy[0][my_index][2])
                      y1=int(results.xyxy[0][my_index][1])
                      y2=int(results.xyxy[0][my_index][3])
                      image_arr2 = image_arr[y1:y2, x1:x2]

                      #resize to 224x224 before put into the model
                      image_arr2 = cv2.resize(image_arr2, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
                      save_img4 = Image.fromarray(image_arr2)
                      save_img4.save("./my_output/crop_alphabet/crop_alphabet"+str(string_count)+".jpg")

                      ### Clasification Model ###
                      img_path = "./my_output/crop_alphabet/crop_alphabet"+str(string_count)+".jpg"
                      stride = classification_model.stride
                      vid_stride=1
                      imgsz=(224, 224)
                      imgsz = check_img_size(imgsz, s=stride)
                      dataset = LoadImages(img_path, img_size=imgsz, transforms=classify_transforms(imgsz[0]), vid_stride=vid_stride)
                      for _, im, _, _, s in dataset:
                        img = im
                      #add batch to image array
                      img = img.unsqueeze(0)
                      #add image array to tensor store in Nvidia vram
                      if(torch.cuda.is_available()):
                        img = torch.Tensor(img).to('cuda')
                      else:
                        img = torch.Tensor(img).to('cpu')

                      img = img.half() if classification_model.fp16 else img.float()
                      if len(img.shape) == 3:
                        img = img[None]
                      #input to model
                      names = classification_model.names
                      classification_results = classification_model(img)
                      pred = F.softmax(classification_results, dim=1)
                      for i, prob in enumerate(pred):
                        top3i = prob.argsort(0, descending=True)[:2].tolist()
                      mark=0
                      for i in top3i:
                        if mark==0:
                          if(prob[i]>=0.1):
                            license_string_list.append(names[i])
                          mark=1
                      letter_index=letter_index+1
                      my_index=count_letter
                      string_count+=1
                    else:
                      letter_index=letter_index+1
                  my_index=my_index+1

              license_len = len(license_string_list)
              check=True

              license_string=""
              for i in license_string_list:
                license_string=license_string+alphabet_dict[i]
              print(license_string,license_len)


              timestr = time.strftime("%Y%m%d-%H%M%S")
              try:
                os.mkdir("./my_output/"+license_string)
                cv2.imwrite('./my_output/'+license_string+"/"+license_string+"-"+timestr+'.jpg',save_img)
                save_img_car.save('./my_output/'+license_string+"/car_"+license_string+"-"+timestr+'.jpg')
              except:
                cv2.imwrite('./my_output/'+license_string+"/"+license_string+"-"+timestr+'.jpg',save_img)
                save_img_car.save('./my_output/'+license_string+"/car_"+license_string+"-"+timestr+'.jpg')


    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

กธ4856 6
กธ4856 6
กธ4856 6
กธ4856 6
กธ4856 6
9กญ719 6
9กญ719 6
9กญ719 6
165029 6
165029 6
165029 6
3กฆ5199 7
3กฆ5199 7
3กฆ5199 7
3กฆ5199 7
4กฌ9728 7
4กฌ9728 7
4กฌ9728 7
ทส4465 6
ทส4465 6
ทส4465 6
ภฐ65 4
ภฐ65 4
ภฐ65 4
ภฐ65 4
กพ7118 6
กพ7118 6
กพ7118 6
กพ7118 6
กง6123 6
กง6123 6
กง6123 6
9กด7411 7
9กด7411 7
9กด7411 7
9กด7411 7
9กด7411 7
9กด7411 7
9กด7411 7
9กด7411 7
9กด7411 7
